In [3]:
!pip install monetdblite
import monetdblite as m
import pandas as pd
from datetime import datetime
m.init('/tmp/MonetDBLite Database')

In [4]:
!wget -O Robust04.rar https://dl.dropboxusercontent.com/s/5qwq3gn6rto98sd/Robust04%2Bpos%2Btf%2Bcf.rar?dl=0
!unrar e -o+ Robust04.rar

--2019-11-18 10:59:46--  https://dl.dropboxusercontent.com/s/5qwq3gn6rto98sd/Robust04%2Bpos%2Btf%2Bcf.rar?dl=0
Resolving dl.dropboxusercontent.com (dl.dropboxusercontent.com)... 162.125.9.6, 2620:100:601f:6::a27d:906
Connecting to dl.dropboxusercontent.com (dl.dropboxusercontent.com)|162.125.9.6|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 460221412 (439M) [application/rar]
Saving to: ‘Robust04.rar’

Robust04.rar        100%[===================>] 438.90M  51.7MB/s    in 8.6s    

2019-11-18 10:59:55 (51.0 MB/s) - ‘Robust04.rar’ saved [460221412/460221412]


UNRAR 5.50 freeware      Copyright (c) 1993-2017 Alexander Roshal


Extracting from Robust04.rar

Extracting  terms.csv                                                      0%  1%  2%  3%  4%  5%  6%  7%  8%  9% 10% 11% 12% 13% 14% 15% 16% 17% 18% 19% 20% 21% 22% 23% 24% 25% 26% 27% 28%

In [0]:
m.sql("CREATE TABLE dict(termid INTEGER "
                       ",term   VARCHAR(99) "
                       ",df     INTEGER "
                       ",cf     INTEGER)")
m.sql("CREATE TABLE terms(termid INTEGER "
                        ",docid  INTEGER "
                        ",pos    INTEGER "
                        ",tf     INTEGER)")
m.sql("CREATE TABLE docs(name  VARCHAR(99) "
                       ",docid INTEGER "
                       ",len   INTEGER "
                       ",temp  INTEGER)") 

In [0]:
m.sql("COPY INTO dict "
      "FROM '/content/dict.csv' "
      "USING DELIMITERS '|'")
m.sql("COPY INTO terms "
      "FROM '/content/terms.csv' "
      "USING DELIMITERS '|'")
m.sql("COPY INTO docs "
      "FROM '/content/docs.csv' "
      "USING DELIMITERS '|'")

In [0]:
def make_query(*args: str):
    """Function to make query table in database.
    
    Args:
        args: concatenation of strings to be made into a query
        
    Returns [str]:
        Number of terms in the query as string.
    """
    query = "('" + args[0] + "')"
    for arg in args[1:]:
        query += ", ('" + arg + "')"
    
    m.sql("DROP TABLE IF EXISTS query")
    m.sql("CREATE TABLE query(term VARCHAR(99))")
    m.sql("INSERT INTO query VALUES " + query)     
        
    return str(len(args))

In [13]:
len_query = make_query('wizard', 'hat')

cur_time = datetime.now()

qterms = m.sql("WITH qtermids "
                    "AS (SELECT dict.termid "
                              ",dict.df "
                        "FROM dict "
                        "JOIN query "
                        "ON dict.term = query.term"
                        ") "
               "SELECT terms.termid "
                     ",terms.docid "
                     ",terms.pos "
                     ",terms.tf "
                     ",qtermids.df "
               "FROM terms "
               "JOIN qtermids "
               "ON terms.termid = qtermids.termid")

time_delta = datetime.now() - cur_time
print('Query time: {}'.format(time_delta))

pd.DataFrame.from_dict(qterms)

Query time: 0:00:00.217486


,termid,docid,pos,tf,df
0,434301,44,589,1,1718
1,434301,152,497,1,1718
2,434301,281,92,1,1718
3,434301,339,1024,1,1718
4,434301,396,696,1,1718
...,...,...,...,...,...
2590,694497,323249,53,2,269
2591,694497,323249,186,2,269
2592,694497,324788,1341,1,269
2593,694497,326675,155,1,269


In [15]:
len_query = make_query('new', 'york')


C = m.sql("SELECT SUM(dict.cf) "
          "FROM dict")
C = str(float(pd.DataFrame.from_dict(C).iloc[0, 0]))
mu = '0.8' # very uncertain of this hyperparameter

N = m.sql("SELECT COUNT(*) "
          "FROM docs")
N = str(pd.DataFrame.from_dict(N).iloc[0, 0])
avgdl = m.sql("SELECT AVG(docs.len) "
              "FROM docs")
avgdl = str(pd.DataFrame.from_dict(avgdl).iloc[0, 0])
k1 = '1.2'
b = '0.75'

k = '20'

maxspan = '5'

cur_time = datetime.now()

t = m.sql("WITH qtermids "
               "AS (SELECT dict.termid "
                         ",dict.df "
                         ",dict.cf "
                   "FROM dict "
                   "JOIN query "
                   "ON dict.term = query.term"
                   ") "
          ", qterms "
               "AS (SELECT terms.termid "
                         ",terms.docid "
                         ",terms.pos "
                         ",terms.tf "
                         ",qtermids.df "
                         ",qtermids.cf "
                   "FROM terms "
                   "JOIN qtermids "
                   "ON terms.termid = qtermids.termid"
                   ") "
          ", condocs "
               "AS (SELECT qterms.docid "
                   "FROM qterms "
                   "GROUP BY qterms.docid "
                   "HAVING COUNT(DISTINCT qterms.termid) = " + len_query +
                   ") "
          ", qtermstf "
               "AS (SELECT qterms.docid "
                         ",qterms.tf "
                         ",qterms.df "
                         ",qterms.cf "
                         ",( ROW_NUMBER() "
                            "OVER(PARTITION BY qterms.termid, qterms.docid "
                                 "ORDER BY qterms.pos"
                                 ")"
                           ") AS row "
                   "FROM qterms "
                  #  "JOIN condocs "
                  #  "ON qterms.docid = condocs.docid "  
                   ") "
          ", okapisubscores "
               "AS (SELECT qtermstf.docid "
                         ",( LOG((" + N + "-df+0.5)/(df+0.5))*tf*(" + k1 + "+1)"
                             "/"
                            "(tf+" + k1 + "*(1-" + b + "+" + b + "*len/" + avgdl + "))"
                           ") AS subscore "
                   "FROM qtermstf "
                   "JOIN docs "
                   "ON qtermstf.docid = docs.docid "
                   "WHERE qtermstf.row = 1"
                   ") "
          ", kldsubscores "
               "AS (SELECT qtermstf.docid "
                         ",( LOG(" + mu + "+tf*" + C + "/cf)"              
                             "+" 
                            "LOG(1/(" + mu + "+len))"
                           ") AS subscore "
                   "FROM qtermstf "
                   "JOIN docs "
                   "ON qtermstf.docid = docs.docid "
                   "WHERE qtermstf.row = 1"
                   ") "
          ", topdocs "
               "AS (SELECT subscores.docid "
                         ",SUM(subscores.subscore) AS score "
                         ",( ROW_NUMBER() "
                            "OVER(ORDER BY SUM(subscores.subscore) DESC)"
                           ") AS row "
                   "FROM okapisubscores AS subscores "
                  #  "FROM kldsubscores AS subscores "
                   "GROUP BY subscores.docid "
                   ") "
          ", topkdocs "
               "AS (SELECT topdocs.docid "
                         ",topdocs.score "
                    "FROM topdocs "
                    "WHERE topdocs.row BETWEEN 1 AND " + k +
                   ") "
          ", pairs "
               "AS (SELECT qterms1.termid AS termid1 "
                         ",qterms2.termid AS termid2 "
                         ",qterms1.docid "
                         ",1.0/(qterms1.pos-qterms2.pos) AS tpi "
                         ",( CASE WHEN qterms1.df > qterms2.df THEN qterms1.df "
                                 "ELSE qterms2.df "
                            "END"
                           ") AS maxdf "
                   "FROM qterms AS qterms1 "
                  #  "JOIN topkdocs "
                  #  "ON qterms1.docid = topkdocs.docid "
                  #  "JOIN condocs "
                  #  "ON qterms1.docid = condocs.docid "
                   "JOIN qterms AS qterms2 "
                   "ON qterms1.docid = qterms2.docid AND "
                      "NOT qterms1.termid = qterms2.termid AND "
                      "qterms1.pos - qterms2.pos BETWEEN 1 AND " + maxspan +
                   ") "
          ",tpisums "
               "AS (SELECT pairs.termid1 "
                         ",pairs.termid2 "
                         ",pairs.docid "
                         ",SUM(pairs.tpi) AS tpisum "
                   "FROM pairs "              
                   "GROUP BY pairs.termid1 "
                           ",pairs.termid2 "
                           ",pairs.docid"
                   ") "
          ", tpsubscores "
               "AS (SELECT pairs.docid "
                         ",( LOG((" + N + "-maxdf+0.5)/(maxdf+0.5))*tpisum*(" + k1 + "+1)"
                             "/"
                            "(tpisum+" + k1 + "*(1-" + b + "+" + b + "*len/" + avgdl + "))"
                           ") AS tpsubscore "
                   "FROM pairs "
                   "JOIN tpisums "
                   "ON pairs.termid1 = tpisums.termid1 AND "
                      "pairs.termid2 = tpisums.termid2 AND "
                      "pairs.docid = tpisums.docid "
                   "JOIN docs "
                   "ON pairs.docid = docs.docid"
                   ") "
          ", tpscores "
               "AS (SELECT tpsubscores.docid "
                         ",SUM(tpsubscores.tpsubscore) AS tpscore "
                   "FROM tpsubscores "
                   "GROUP BY tpsubscores.docid"
                   ") "
          ", scores "
               "AS (SELECT topkdocs.docid "
                         ",( topkdocs.score"
                             "+"
                            "COALESCE(tpscores.tpscore, 0)"
                           ") AS score "
                   "FROM topkdocs "
                   "LEFT JOIN tpscores "
                   "ON topkdocs.docid = tpscores.docid"
                   ") "
          "SELECT scores.docid "
                ",scores.score "
          "FROM scores "
          "ORDER BY scores.score DESC ")

time_delta = datetime.now() - cur_time
print('Query time: {}'.format(time_delta))

pd.DataFrame.from_dict(t)

Query time: 0:00:00.779491


,docid,score
0,50961,137.205792
1,84012,113.877312
2,18841,60.242891
3,283220,48.525844
4,51576,47.291489
5,118444,46.209836
6,316964,45.946838
7,100831,42.419597
8,198684,40.250488
9,314396,37.078993
